## 1. 1번 데이터셋 전처리 코드

In [ ]:
import pandas as pd

In [ ]:
# 경로 자기 파일이랑 맞게 바꿔주자.

data_2018 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/2018년.xlsx')
data_2019 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/2019년.xlsx')
data_2020 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/ 2020년.xlsx')
data_2021 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/2021년.xlsx')
data_2022 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/2022년.xlsx')


# 공공데이터 스케줄
matching_data_21 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/한국공항공사_국내선항공기스케줄_20210910.xlsx')
matching_data_22 = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/한국공항공사_국내선 항공기스케줄_20220919.xlsx')


# 항공 기종 매칭 데이터
matching_korea_airplane = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/9. 기종별 매칭.xlsx')
# 항공사별 항공기 정보
airplane_info = pd.read_excel('/Users/kanovatio/Gangmin Jeong/대학/3-2/디지털경영론/중간고사 프로젝트/디지털경영론 data/데이터/데이터셋/7. 항공사별 항공기 정보.xlsx')

# 조인 시켜주기 위해 ICAO 열을 일단 기종으로 변경
airplane_info.rename({'ICAO':'기종'}, axis=1, inplace=True)


# 기종 매칭
data_2021 = pd.merge(data_2021, matching_data_21, how='left',on='운항편명')
data_2022 = pd.merge(data_2022, matching_data_22, how='left',on='운항편명')

# 데이터 없는 나머지는 용량이 많은 2021로 조인해주자
data_2018 = pd.merge(data_2018, matching_data_21, how='left',on='운항편명')
data_2019 = pd.merge(data_2019, matching_data_21, how='left',on='운항편명')
data_2020 = pd.merge(data_2020, matching_data_21, how='left',on='운항편명')

In [ ]:
def process(data):
    data.drop(data[data['항공사명']=='하이에어'].index, inplace=True)
    data.drop(data[data['항공사명']=='에어프레미아'].index, inplace=True)
    data.drop(data[data['항공사명']=='에어필립'].index, inplace=True)

    # NaN 전부 보내주자
    data.dropna(inplace=True)

    # 인덱스 초기화
    data.reset_index(drop=True, inplace=True)

    # 애매한 친구들 바꿔주자
    data['기종'] = data['기종'].apply(lambda x: 'BCS3' if x == 'A220' else x)
    data['기종'] = data['기종'].apply(lambda x: 'B763' if x == 'B767' else x)

    # 제주공항 -> CJU ,김포공항 -> GMP
    data['출발공항'] = data['출발공항'].apply(lambda x: 'GMP' if x == '서울/김포' else 'CJU')
    data['도착공항'] = data['출발공항'].apply(lambda x: 'GMP' if x == '서울/김포' else 'CJU')


    # 운항시간 계산 전에 시간을 int형으로 변환
    data['출발예정시간'] = data['출발예정시간'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1]))
    data['도착예정시간'] = data['도착예정시간'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1]))

     # 실제 운항시간 계산
    data['운항시간'] = data['도착예정시간']//100*60 + data['도착예정시간']%100 - data['출발예정시간']//100*60 - data['출발예정시간']%100

    # 기종명 매칭
    code_to_model = dict([(key,value) for key, value in zip(matching_korea_airplane['기종 1'], matching_korea_airplane['기종 2'])])
    data['기종명'] = data['기종'].map(lambda x:code_to_model[x])


    # 항공사명 -> 항공사로 열 이름 변경
    data.rename({'항공사명':'항공사'}, axis=1, inplace=True)

    data = pd.merge(data, airplane_info, how='left', on=['항공사','기종명']).drop(['기종_x', '기종_y'], axis=1)

    data.rename({'출발예정시간':'출발시간','도착예정시간':'도착시간', '기종명':'기종'}, axis=1, inplace=True)

    return data

In [ ]:
# 함수 실행
data_2018 = process(data_2018)
data_2019 = process(data_2019)
data_2020 = process(data_2020)
data_2021 = process(data_2021)
data_2022 = process(data_2022)

In [ ]:
data_2018.head()

,운항편명,항공사,출발공항,도착공항,출발시간,도착시간,운항일자,운항시간,기종,제조사,좌석수,길이(m),날개 폭(m),높이(m),최대운항거리(km),순항속도(km/h)
0,KE1233,대한항공,GMP,CJU,1355,1505,2018.06.01,70,A330-300,Airbus,272,63.69,60.3,16.83,9449,879
1,KE1245,대한항공,GMP,CJU,1600,1710,2018.06.01,70,B737-700,Boeing,126,33.60,35.8,12.60,8149,946
2,KE1207,대한항공,GMP,CJU,830,940,2018.06.01,70,B737-700,Boeing,126,33.60,35.8,12.60,8149,946
3,KE1207,대한항공,GMP,CJU,830,940,2018.06.01,70,B737-700,Boeing,126,33.60,35.8,12.60,8149,946
4,KE1207,대한항공,GMP,CJU,830,940,2018.06.01,70,A330-300,Airbus,272,63.69,60.3,16.83,9449,879


In [ ]:
data_2018.to_excel('data_2018.xlsx')

In [ ]:
data_2019.to_excel('data_2019.xlsx')

In [ ]:
data_2020.to_excel('data_2020.xlsx')

In [ ]:
data_2021.to_excel('data_2021.xlsx')

In [ ]:
data_2022.to_excel('data_2022.xlsx')